In [1]:
import os
import math
import time
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import fishUtilities as fu
import fishFits as ff
import surfaceAreaEstimators as sea

In [2]:
imagePath = r'.\Pictures\powerThrustData\UV Tunabot'
figurePath = r'.\Pictures\powerThrustData\UV Tunabot'

specimen = 'UVTunabot'

tLength = 0.255
sLength = 0.215

In [4]:
imagePathTop = fu.get_image_files(imagePath, specimen+'Top.PNG')
imagePathSide = fu.get_image_files(imagePath, specimen+'SideNF.PNG')
imagePathSideFull = fu.get_image_files(imagePath, specimen+'Side.PNG')
    
# import images
imageTop = cv2.imread(imagePathTop[0])
imageSide = cv2.imread(imagePathSide[0])
imageSideFull = cv2.imread(imagePathSideFull[0])
    
# make a copy of the image
sideImage = imageSide.copy()
topImage = imageTop.copy()
sideImageFull = imageSideFull.copy()
        
# get contours
cntsTop, hullTop = fu.get_contours(imageTop, 1)
cntsSide, hullSide = fu.get_contours(imageSide, 1)
cntsFull, hullFUll = fu.get_contours(imageSideFull, 1)
    
# determine the most extreme points along the contour
extSideLeft = tuple(cntsSide[cntsSide[:, :, 0].argmin()][0])
extSideRight = tuple(cntsSide[cntsSide[:, :, 0].argmax()][0])
extTopLeft = tuple(cntsTop[cntsTop[:, :, 0].argmin()][0])
extTopRight = tuple(cntsTop[cntsTop[:, :, 0].argmax()][0])
extTop = tuple(cntsFull[cntsFull[:, :, 1].argmin()][0])
extBot = tuple(cntsFull[cntsFull[:, :, 1].argmax()][0])
    
# Find the area ratio of fins and no fins
areaFull = cv2.contourArea(cntsFull)
areaNF = cv2.contourArea(cntsSide)
areaRatio = (1-np.abs(areaNF/areaFull))
    
# split the hull into top and bottom
(cxTop, cyTop), topHullTop, bottomHullTop = fu.split_by_centroid_top(imageTop.shape, cntsTop)
(cxSide, cySide), areaSide, topHullSide, bottomHullSide = fu.split_by_centroid_side(imageSide.shape, cntsSide)
(cxFull, cyFull), areaSideFull, topHullSideFull, bottomHullSideFull = fu.split_by_centroid_side(imageSideFull.shape, cntsFull)
    
# get contour data
areaTop, tHullTop, bHullTop = fu.scale_data_top(topHullTop, bottomHullTop)
scaleFactorSide, tHullSide, bHullSide = fu.scale_data(topHullSide, bottomHullSide)
scaleFactorSideFull, tHullSideFull, bHullSideFull = fu.scale_data(topHullSideFull, bottomHullSideFull)
        
#areaTopRatio = 1-(areaTop/areaTopFull)
# get min and max points
maxPointsTop, minPointsTop = fu.get_min_max(tHullTop, bHullTop)
maxPointsSide, minPointsSide = fu.get_min_max(tHullSide, bHullSide)
        
dx = np.linspace(0, 1.0, 200)
# fit data
topPolyArea, topFitTop, bottomFitTop = ff.fit_top_contours_poly(tHullTop, bHullTop, 6)
topFitSide, bottomFitSide = ff.fit_side_contours(tHullSide, bHullSide, 6)
nacaArea, m, thickness, d0, xu, yu, xl, yl = ff.fit_top_contours(areaTop, np.mean([minPointsTop[0],maxPointsTop[0]]),
                                                 np.abs(minPointsTop[1])+maxPointsTop[1], (tHullTop[1,-5]+np.abs(bHullTop[1,-5]))/2.0)
topPolyTop = np.poly1d(topFitTop)
bottomPolyTop = np.poly1d(bottomFitTop)
topPolySide = np.poly1d(topFitSide)
bottomPolySide = np.poly1d(bottomFitSide)
        
    
nacaSA = sea.surface_area_fusiform(topPolySide, bottomPolySide, xu, yu, sLength)
nacaSA = nacaSA + (nacaSA*areaRatio)
polySA = sea.surface_area_oval(dx, topPolySide, bottomPolySide, topPolyTop, bottomPolyTop, sLength)
polySA = polySA + (polySA*areaRatio)

In [5]:
print(m, thickness, d0)
print(topFitSide, bottomFitSide)

0.4277474195506982 0.2067378287017953 0.010625379477838493
[-6.75527413e+00  2.23575120e+01 -2.77639294e+01  1.64262105e+01
 -5.46089204e+00  1.19166754e+00  1.29582527e-02] [ 5.55222913e+00 -1.78299369e+01  2.15461223e+01 -1.26445185e+01
  4.50657048e+00 -1.13482382e+00 -6.95468036e-03]


In [6]:
nacaSA

0.02941724283035747